In [1]:
#no metrics
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np
import cv2
from scipy import ndimage
from skimage import morphology
#from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score, f1_score
from tensorflow.keras.models import load_model
from metrics import dice_coef, iou

# Load pre-trained models
unet_model = load_model('/Users/shravanisajekar/Downloads/unet_model.h5', custom_objects={'dice_coef': dice_coef, 'iou': iou})
hrnet_model = load_model('/Users/shravanisajekar/Downloads/27-3-2024 UPDATE/weight_isic18', compile=False)
segnet_model = load_model('/Users/shravanisajekar/Downloads/segnet_10_epoch.h5', compile=False)

# Define the predict_mask function
def predict_mask(filename):
    try:
        image = Image.open(filename)
    
        # Resize the image
        image_hrnet = image.resize((256, 256))
        image_segnet = image.resize((256, 192))

        # Define weights for each model
        unet_weight = 0.3
        hrnet_weight = 0.6
        segnet_weight = 0.5

        # Make predictions using all three models
        unet_pred = unet_model.predict(np.expand_dims(np.array(image_hrnet), axis=0)) * unet_weight
        hrnet_pred = hrnet_model.predict(np.expand_dims(np.array(image_hrnet), axis=0)) * hrnet_weight
        segnet_pred = segnet_model.predict(np.expand_dims(np.array(image_segnet), axis=0)) * segnet_weight

        # Resize SegNet predictions to match the shape of UNet and HRNet predictions
        segnet_pred_resized = cv2.resize(segnet_pred[0], (256, 256))

        # Combine predictions with weights
        ensemble_pred = (unet_pred + hrnet_pred + segnet_pred_resized) / (unet_weight + hrnet_weight + segnet_weight)
        
        # Convert combined mask to single-channel image
        combined_mask = ((ensemble_pred > 0.5) * 255).astype(np.uint8)
        combined_mask_single_channel = combined_mask.squeeze() 

        # Ensure that the mask has shape (height, width)
        if len(combined_mask_single_channel.shape) > 2:
            combined_mask_single_channel = combined_mask_single_channel[:, :, 0]
                
        # Adjust brightness and contrast of the prediction
        brightness_factor = 50
        contrast_factor = 1.5
        combined_mask_single_channel = combined_mask_single_channel * contrast_factor + brightness_factor
        combined_mask_single_channel = np.clip(combined_mask_single_channel, 0, 255).astype(np.uint8)
        return Image.fromarray(combined_mask_single_channel, mode='L') 
    
    except Exception as e:
        print("Error predicting mask:", e)
        return None

      
def show_images(img, mask):
    
    # Resize input image to match dimensions of mask
    img = img.resize(mask.size)

    img_tk = ImageTk.PhotoImage(img)
    mask_tk = ImageTk.PhotoImage(mask)

    mask_window = tk.Toplevel(root)
    mask_window.title("Input and Mask Prediction")

    input_label = tk.Label(mask_window, image=img_tk)
    input_label.image = img_tk
    input_label.grid(row=0, column=0)

    mask_label = tk.Label(mask_window, image=mask_tk)
    mask_label.image = mask_tk
    mask_label.grid(row=0, column=1)       

    # Create a label for displaying metrics
    global label
    label = tk.Label(mask_window,text='NOTE: ', font=("Helvetica", 14))
    label1 = tk.Label(mask_window,text='- If the output is white, no lesion is detected!!', font=("Helvetica", 14))
    label2 = tk.Label(mask_window,text='- If dark spot is seen after segmentation, please consult the doctor!!', font=("Helvetica", 14))
    label.grid(row=1, columnspan=2)
    label1.grid(row=2, columnspan=3)
    label2.grid(row=3, columnspan=4)


def refine_mask(mask):

    mask_smoothed = morphology.binary_closing(mask)
    mask_smoothed = morphology.binary_opening(mask_smoothed)
    
    return mask_smoothed

# Create the GUI
if __name__ == "__main__":
    root = tk.Tk()
    root.title("Skin Lesion Predictor")
    bg_image_path = "/Users/shravanisajekar/Downloads/27-3-2024 UPDATE/skin_cancer_1200x540.jpg"
    bg_image = Image.open(bg_image_path)
    width, height = bg_image.size
    new_width = min(width, height)
    root.geometry(f"{new_width}x{new_width}")

    # Convert image to PhotoImage
    bg_photo = ImageTk.PhotoImage(bg_image)

    # Create a label to display the background image
    bg_label = tk.Label(root, image=bg_photo)
    bg_label.place(x=0, y=0)

    title_label = tk.Label(root, text="Skin Lesion Segmentation", font=("Times New Roman", 24, "underline"), fg="black", bg='#ffb8b8')
    title_label.pack(pady=20)

    # Function to browse for an image
    def browse_image():
        filename = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg")])
        if filename:
            img = Image.open(filename)
            mask = predict_mask(filename)
            if mask:
                show_images(img, mask)
            else:
                print("Error: Unable to predict mask.")

    browse_button = tk.Button(root, text="Browse Image", command=browse_image, font=("Helvetica", 14), bg='white', highlightbackground='#ffb8b8', highlightcolor='white')
    browse_button.place(relx=0.2, rely=0.5, anchor='s')

    root.mainloop()
